In [8]:
import numpy as np
import string

In [97]:
def TupleToString(tup):
    # Convert a tuple to a string where each number is assigned to its string form.
    return "".join(list(map(lambda i: string.printable[10 + i], tup)))
TupleToString((1,2))

'bc'

In [88]:
def YieldScalar(tndict):
    # Check if contracting all the tensors in the dictionary yields a scalar.
    supports = np.array([np.array(list(support), dtype = np.int) for (support, __) in tndict], dtype = np.int).flatten()
    #print("supports = {}".format(supports))
    visited = np.zeros(supports.shape[0], dtype = np.int)
    for i in range(supports.size):
        not_scalar = 1
        for j in range(supports.size):
            if (i != j):
                if (supports[i] == supports[j]):
                    not_scalar = 0
                    #print("Comparing support[%d] = %d and support[%d] = %d." % (i, supports[i], j, supports[j])) 
    scalar = 1 - not_scalar
    return scalar

In [99]:
def TensorTrace(tndict, opt = "greedy"):
    # Contract a set of tensors provided as a dictionary.
    # The result of the traction must be a number, in other words, there should be no free indices.
    if (YieldScalar(tndict) == 0):
        print("The Tensor contraction does not result in a trace.")
        return None
    # We want to use np.einsum(...) to perform the Tensor contraction.
    # Every element of the input dictionary is a tuple, containing the support of a tensor and its operator form.
    # Numpy's einsum function simply needs the Tensors and the corresponding labels associated to its indices.
    # We will convert the support of each Tensor to a string, that serves as its label.
    scheme = ",".join([TupleToString(support) for (support, __) in tndict])
    print("Contraction scheme: {}".format(scheme))
    ops = [op for (__, op) in tndict]
    ops_args = ", ".join([("ops[%d]" % d) for d in range(len(ops))])
    print("np.einsum_path(\'%s->\', %s, optimize=\'%s\')" % (scheme, ops_args, opt))
    path = eval("np.einsum_path(\'%s->\', %s, optimize=\'%s\')" % (scheme, ops_args, opt))
    print("Contraction process\n{}: {}\n{}".format(path[0][0], path[0][1:], path[1]))
    trace = np.einsum(scheme, *ops, optimize=path[0])
    #print("Trace = {}.".format(trace))
    return trace

In [100]:
N = 24
dims = 4
bond = 4
# create some numpy tensors
tensors = [np.random.rand(*[bond]*dims) for __ in range(0, N, 2)]
# labels should ensure that there is no free index.
labels = [tuple([(i + j) % N for j in range(4)]) for i in range(0, N, 2)]
print(labels)
# Prepare the dictionary.
tndict = [(labels[i], tensors[i]) for i in range(len(labels))]
#print([tnop for (lab, tnop) in tndict])

[(0, 1, 2, 3), (2, 3, 4, 5), (4, 5, 6, 7), (6, 7, 8, 9), (8, 9, 10, 11), (10, 11, 12, 13), (12, 13, 14, 15), (14, 15, 16, 17), (16, 17, 18, 19), (18, 19, 20, 21), (20, 21, 22, 23), (22, 23, 0, 1)]


In [ ]:
trace = TensorTrace(tndict, opt="optimal")
print("Trace = {}.".format(trace))

Contraction scheme: abcd,cdef,efgh,ghij,ijkl,klmn,mnop,opqr,qrst,stuv,uvwx,wxab
np.einsum_path('abcd,cdef,efgh,ghij,ijkl,klmn,mnop,opqr,qrst,stuv,uvwx,wxab->', ops[0], ops[1], ops[2], ops[3], ops[4], ops[5], ops[6], ops[7], ops[8], ops[9], ops[10], ops[11], optimize='optimal')


In [182]:
def OptimalEinsum(scheme, ops, opt = "greedy"):
    # Contract a tensor network using einsum supplemented with its optimization tools.
    ops_args = ", ".join([("ops[%d]" % d) for d in range(len(ops))])
    print("Calling np.einsum({}, {}).".format(scheme, ops_args))
    path = eval("np.einsum_path(\'%s\', %s, optimize=\'%s\')" % (scheme, ops_args, opt))
    print("Contraction process\n{}: {}\n{}".format(path[0][0], path[0][1:], path[1]))
    prod = np.einsum(scheme, *ops, optimize=path[0])
    return prod

In [183]:
def TensorTranspose(tensor):
    # Transpose the tensor, in other words, exchange its row and column indices.
    # Note that when we reshape a matrix into (D, D, ..., D) tensor, it stores the indices as as
    # row_1, row_2, row_3, ..., row_(D/2), col_1, ..., col_(D/2).
    rows = range(0, tensor.ndim//2)
    cols = range(tensor.ndim//2, tensor.ndim)
    tp_indices = np.concatenate((cols, rows))
    return np.transpose(tensor, tp_indices)

In [159]:
# Testing TensorTranspose
nq = 10
tensor = np.reshape(np.random.rand(2**nq, 2**dims), [2, 2]*nq)
tp_tensor = TensorTranspose(tensor)
#print("tensor\n{}\nand its transpose\n{}".format(tensor.reshape(2**dims, 2**dims), tp_tensor.reshape(2**dims, 2**dims)))
print("A ?= A.T is {}".format(np.allclose(tensor.reshape(2**nq, 2**nq), tp_tensor.reshape(2**nq, 2**nq).T)))

A ?= A.T is True


In [138]:
def GetNQubitPauli(ind, nq):
    # Compute the n-qubit Pauli that is at position 'i' in an ordering based on [I, X, Y, Z].
    # We will express the input number in base 4^n - 1.
    pauli = np.zeros(nq, dtype = np.int)
    for i in range(nq):
        pauli[i] = ind % 4
        ind = int(ind//4)
    return pauli

In [160]:
# Testing GetNQubitPauli
GetNQubitPauli(172, 4)

array([0, 3, 2, 2])

In [217]:
def TensorKron(tn1, tn2):
    # Compute the Kronecker product of two tensors A and B.
    # This is not equal to np.tensordot(A, B, axis = 0), see: https://stackoverflow.com/questions/52125078/why-does-tensordot-reshape-not-agree-with-kron.
    # Note that when we reshape a matrix into (D, D, ..., D) tensor, it stores the indices as as
    # row_1, row_2, row_3, ..., row_(D/2), col_1, ..., col_(D/2).
    # We will implement Kronecker product using einsum, as
    # np.einsum('rA1 rA2 .. rAn, cA1 cA2 .. cAn, rB1 rB2 .. rBn, cB1 cB2 .. cBn -> rA1 rB1 rA2 rB2 .. rAn rBn, cA1 cB1 cA2 cB2 .. cAn cBn', A, B).
    if (tn1.ndim != tn2.ndim):
        print("TensorKron does not work presently for tensors of different dimensions.")
        return None
    tn1_rows = [string.printable[10 + i] for i in range(tn1.ndim//2)]
    tn1_cols = [string.printable[10 + i] for i in range(tn1.ndim//2, tn1.ndim)]
    tn2_rows = [string.printable[10 + tn1.ndim + i] for i in range(tn2.ndim//2)]
    tn2_cols = [string.printable[10 + tn1.ndim + i] for i in range(tn2.ndim//2, tn2.ndim)]
    #kron_inds = ["%s%s" % (tn1_rows[i], tn2_rows[i]) for i in range(tn1.ndim//2)]
    #kron_inds += ["%s%s" % (tn1_cols[i], tn2_cols[i]) for i in range(tn1.ndim//2)]
    kron_inds = ["%s" % (tn1_rows[i]) for i in range(tn1.ndim//2)]
    kron_inds += ["%s" % (tn2_rows[i]) for i in range(tn1.ndim//2)]
    kron_inds += ["%s" % (tn1_cols[i]) for i in range(tn1.ndim//2)]
    kron_inds += ["%s" % (tn2_cols[i]) for i in range(tn1.ndim//2)]
    scheme = ("%s%s,%s%s->%s" % ("".join(tn1_rows), "".join(tn1_cols), "".join(tn2_rows), "".join(tn2_cols), "".join(kron_inds)))
    return OptimalEinsum(scheme, [tn1, tn2], opt = "greedy")

In [218]:
# Testing TensorKron
nq = 4
tn1 = np.reshape(np.random.rand(2**nq, 2**nq), [2, 2]*nq)
tn2 = np.reshape(np.random.rand(2**nq, 2**nq), [2, 2]*nq)
kn_tensor = TensorKron(tn1, tn2)
#print("A\n{}\nB\n{}\nA o B = \n{}".format(tn1.reshape(2**nq, 2**nq), tn2.reshape(2**nq, 2**nq), kn_tensor.reshape(4**nq, 4**nq)))
print("TensorKron(A, B) ?= A o B is {}".format(np.allclose(kn_tensor.reshape(4**nq, 4**nq), np.kron(tn1.reshape(2**nq, 2**nq), tn2.reshape(2**nq, 2**nq)))))

Calling np.einsum(abcdefgh,ijklmnop->abcdijklefghmnop, ops[0], ops[1]).
Contraction process
einsum_path: [(0, 1)]
  Complete contraction:  abcdefgh,ijklmnop->abcdijklefghmnop
         Naive scaling:  16
     Optimized scaling:  16
      Naive FLOP count:  6.554e+04
  Optimized FLOP count:  6.554e+04
   Theoretical speedup:  1.000
  Largest intermediate:  6.554e+04 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
  16    ijklmnop,abcdefgh->abcdijklefghmnop       abcdijklefghmnop->abcdijklefghmnop
TensorKron(A, B) ?= A o B is True


In [261]:
def TraceDot(tn1, tn2):
    # Compute the trace of the dot product of two tensors A and B.
    # If the indices of A are i_0 i_1, ..., i_(2n-1) and that of B are j_0 j_1 ... j_(2n-1)
    # then want to contract the indices i_(2k) with j_(2k+1), for all k in [0, n-1].
    # While calling np.einsum, we need to ensure that the row index of A is equal to the column index of B.
    # Additionally to ensure that we have a trace, we need to match the row and column indices of the product.
    tn1_rows = [string.printable[10 + i] for i in range(tn1.ndim//2)]
    tn1_cols = [string.printable[10 + i] for i in range(tn1.ndim//2, tn1.ndim)]
    # The column indices of tn1 should match row indices of tn2
    # So, tn1_cols = tn2_rows.
    # the row and column indices of the product must match
    # So, tn1_rows = tn2_cols.
    scheme = ("%s%s,%s%s->" % ("".join(tn1_rows), "".join(tn1_cols), "".join(tn1_cols), "".join(tn1_rows)))
    return OptimalEinsum(scheme, [tn1, tn2], opt = "greedy")

In [283]:
# Testing TraceDot
nq = 5
tn1 = np.reshape(np.random.rand(2**nq, 2**nq), [2, 2]*nq)
tn2 = np.reshape(np.random.rand(2**nq, 2**nq), [2, 2]*nq)
trdot = TraceDot(tn1, tn2)
nptr = np.trace(np.dot(tn1.reshape(2**nq, 2**nq), tn2.reshape(2**nq, 2**nq)))
#print("A\n{}\nB\n{}\nTr(A . B) = \n{}".format(tn1.reshape(2**nq, 2**nq), tn2.reshape(2**nq, 2**nq), trdot))
#print("TensorTrace = {}\nNumpy Trace = {}".format(trdot, nptr))
print("TraceDot(A, B) ?= Tr(A . B) is {}.".format(np.allclose(trdot, nptr)))

Calling np.einsum(abcdefghij,fghijabcde->, ops[0], ops[1]).
Contraction process
einsum_path: [(0, 1)]
  Complete contraction:  abcdefghij,fghijabcde->
         Naive scaling:  10
     Optimized scaling:  10
      Naive FLOP count:  2.048e+03
  Optimized FLOP count:  2.049e+03
   Theoretical speedup:  1.000
  Largest intermediate:  1.000e+00 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
  10     fghijabcde,abcdefghij->                                       ->
TraceDot(A, B) ?= Tr(A . B) is True.


In [141]:
def KraussToTheta(kraus):
    # Convert from the Kraus representation to the "Theta" representation.
    # The "Theta" matrix T of a CPTP map whose chi-matrix is X is defined as:
    # T_ij = \sum_(ij) [ X_ij (P_i o (P_j)^T) ]
    # Note that the chi matrix X can be defined using the Kraus matrices {K_k} in the following way
    # X_ij = \sum_k [ <P_i|K_k><K_k|P_j> ]
    # 	   = \sum_k [ Tr(P_i K_k) Tr((K_k)^\dag P_j)]
    # So we find that
    # T = \sum_(ij) [ \sum_k [ Tr(P_i K_k) Tr((K_k)^\dag P_j)] ] (P_i o (P_j)^T) ]
    # We will store T as a Tensor with dimension = (2 * number of qubits) and bond dimension = 4.
    nq = kraus.shape[0]
    theta = np.array(*[4, 4]*nq, dtype = np.complex128)
    for i in range(4**nq):
        for j in range(4**nq):
            Pi = get_Pauli_tensor(GetNQubitPauli(i))
            Pj = GetNQubitPauli(j)
            PjT = TensorTranspose(get_Pauli_tensor(Pj))
            PioPjT = TensorKron(Pi, PjT)
            coeff = 0 + 0 * 1j
            for k in range(kraus.shape[0]):
                K = np.reshape(kraus[k, :, :], *[2, 2]*nq)
                Kdag = np.conj(TensorTranspose(K))
                coeff += TraceDot(Pi, K) * TraceDot(Pj, Kdag)
            theta += coeff * PioPjT
    return theta